In [ ]:
import numpy as np
import cv2
import mediapipe as mp
from fiftyone.utils.transformers import torch
from mediapipe.tasks import python
from mediapipe.tasks.python import vision


Read in

In [ ]:
img_path = "./data/outputs/spill01_seg_img.png"
image_bgr = cv2.imread(img_path)

# Convert to HSV and split
img_hsv = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2HSV)
h, s, v = cv2.split(img_hsv)
h = h.astype(np.float32) / 180.0    # Normalise hue to [0.0, 1.0]

# Convert to RGB and split
img_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0
r, g, b = cv2.split(img_rgb)

# Compute greenness: G - max(R, B), then clip to [0.0, 1.0]
greenness = g - np.maximum(r, b)
greenness = np.clip(greenness, 0.0, 1.0)

input_tensor = np.stack([r, g, b, h, greenness], axis=-1)   # Shape: (H, W, 5), dtype: float32
img_tensor = torch.tensor(input_tensor).permute(2, 0, 1).unsqueeze(0)   # Shape: (1, 5, H, W)
